# Python - Pandas  

[Pandas Reference](http://pandas.pydata.org/pandas-docs/stable/reference/index.html)

## Series and DataFrames

In [ ]:
import numpy as np
import pandas as pd

- Series

In [ ]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

- DataFrames # Very similar to R dataframes

In [ ]:
import pandas as pd
dates = pd.date_range('20130101', periods=6) 
# R + lubridate:  dates <- ymd("2013-01-01")+days(0:5)
dates

In [ ]:
df = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
# R + tidyverse: 
# m <- matrix(rnorm(6*4), ncol = 4)
# colnames(m) <- c("A", "B", "C", "D")
# df <- as_tibble(m) %>% mutate(dates)
df

## Pandas vs. R

In [ ]:
df.shape # R: dim(df)

In [ ]:
df.head(3) # R: head(df, 3)

In [ ]:
df.mean()
# R + tidyverse: df %>% select_if(is.numeric) %>% map_dbl(mean, na.rm = TRUE)

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
sns.pairplot(df[["A", "B", "C"]])
plt.show()
# R + GGally: df %>% select(A, B, C) %>% ggpairs()

## Pandas vs. Tidyr

| **pandas** | **tidyr** |
| :---: | :---: |
| melt |	gather |
| pivot |	spread |

## Pandas vs. Dplyr
| **pandas** | **dplyr** |
| :---: | :---: |
| mutate |	assign |
| select |	filter |
| rename |	rename |
| filter |	query |
| arrange |	sort_values |
| group_by |	groupby |
| summarize |	agg 


## Panda read data from Kaggle by using Kaggle's API
[GitHub reference](https://github.com/kaggle/kaggle-api)

It is required the package `kaggle`
        #%%bash
        pip install kaggle --upgrade

Then it is required to **'Create API Token'** from the 
Kraggle Account.

This will trigger the download of `kaggle.json`, 
a file containing your API credentials. 

Place this file in the location `~/.kaggle/kaggle.json`

In [ ]:
import kaggle
import pandas as pd
kaggle.api.authenticate()
kaggle.api.dataset_download_files('toramky/automobile-dataset', unzip=True)
car_data = pd.read_csv("Automobile_data.csv")

## Panda read data from url 

In [ ]:
import pandas as pd
import io
import requests as rq

url="https://think.cs.vt.edu/corgis/csv/cars/cars.csv?forcedownload=1"
df=pd.read_csv((io.StringIO(rq.get(url).content.decode('utf-8'))))

we start by visualizing the variables, the number of observations and some of them

In [ ]:
from IPython.core.display import display, HTML

print(df.columns)
display(HTML('<div>Number of observation: <font style="color: yellow;">%d</span></font>' % len(df)))
df.head(2)

In [ ]:
df['Model Year'].unique()[:5]

Let us define a function to extract the _car model_ from the column `Model Year`

In [ ]:
def reduce_model_year(my,y):
    return my.replace(str(y),'');

The we use the `DataFrame` method `apply` to crate the new column `model`

In [ ]:
if 'model' in df.columns:       # if the column `model` has been created before
    df.drop('model', axis=1)    # drop it
df['model'] = df.apply(lambda x: reduce_model_year(x['Model Year'], x['Year']), axis=1)
df.filter(items=['Model Year', 'model']).head(3)

An alternative way to make this would be by using the `assign` method together with the `numpy.vectorize`
function that is used to vectoring the function `reduce_model_year`

In [ ]:
import numpy as np
if 'model' in df.columns:       # if the column `model` has been created before
    df.drop('model', axis=1)    # drop it
df = df.assign(model=lambda x: np.vectorize(reduce_model_year)(x['Model Year'], x['Year']), axis=1)
df[['Model Year', 'model']].head(3) # other way to make a filter

Let us count the number of lines containing `NaNs`value

In [ ]:
len(df) - df.count()

### Using Group-by’s and Merges

In [ ]:
group_by_model = df.groupby(by= df['model'])

In [ ]:
df_count = (group_by_model 
 .agg({'ID':'count', 'Model Year':'first'})
 .rename(columns={'ID':'Count'}))
df_count.head()

In [ ]:
df_mean = group_by_model.mean()
df_mean.head()

In [ ]:
df_merge = df_mean.merge(df_count, left_index=True, right_index=True, suffixes=['_avg','_count'])

In [ ]:
df_merge[['City mpg', 'Count']].sort_values(by=['Count','City mpg'], ascending=False).head(5).plot(kind='barh')